In [75]:
import re
import numpy as np

import polars as pl
import csv

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [36]:
file_paths = {
    "test_arabic_negative_tweets": "../data/test_arabic_negative_tweets.tsv",
    "test_arabic_positive_tweets": "../data/test_arabic_positive_tweets.tsv",
    "train_arabic_negative_tweets": "../data/train_arabic_negative_tweets.tsv",
    "train_arabic_positive_tweets": "../data/train_arabic_positive_tweets.tsv",
}

In [41]:
def load_data(file_path: str, label: int) -> pl.DataFrame:
    """
    Load data from a TSV file and assign a label to each row.

    Args:
        file_path (str): The path to the TSV file.
        label (int): The label to assign to each tweet (0 for negative, 1 for positive).

    Returns:
        pl.DataFrame: A Polars DataFrame containing tweets and their labels.
    """
    rows = []
    with open(file_path, newline="", encoding="utf-8") as tsvfile:
        reader = csv.reader(tsvfile, delimiter="\t")
        for row in reader:
            rows.append([row[1], label])
    return pl.DataFrame(rows, schema=["tweet", "label"])


def preprocess_text(text: str) -> str:
    """
    Preprocess a tweet by removing mentions, URLs, punctuation, and extra spaces.

    Args:
        text (str): The tweet text to preprocess.

    Returns:
        str: The cleaned tweet text.
    """
    text = re.sub(r"@\w+", "", text)  # Remove mentions
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

In [44]:
train_neg = load_data(file_paths["train_arabic_negative_tweets"], 0)
train_pos = load_data(file_paths["train_arabic_positive_tweets"], 1)
test_neg = load_data(file_paths["test_arabic_negative_tweets"], 0)
test_pos = load_data(file_paths["test_arabic_positive_tweets"], 1)

train_df = pl.concat([train_neg, train_pos])
test_df = pl.concat([test_neg, test_pos])
df = pl.concat([train_df, test_df])
df.head()

tweet,label
str,i64
"""اعترف ان بتس كانو شوي شوي يجيب…",0
"""توقعت اذا جات داريا بشوفهم كام…",0
"""#الاهلي_الهلال اكتب توقعك لنتي…",0
"""نعمة المضادات الحيوية . تضع قط…",0
"""الدودو جايه تكمل علي 💔""",0


In [46]:
df = df.with_columns(
    [
        pl.col("tweet")
        .map_elements(preprocess_text, return_dtype=str)
        .alias("cleaned_tweet")
    ]
)

tweet,label,cleaned_tweet
str,i64,str
"""اعترف ان بتس كانو شوي شوي يجيب…",0,"""اعترف ان بتس كانو شوي شوي يجيب…"
"""توقعت اذا جات داريا بشوفهم كام…",0,"""توقعت اذا جات داريا بشوفهم كام…"
"""#الاهلي_الهلال اكتب توقعك لنتي…",0,"""الاهلي_الهلال اكتب توقعك لنتيج…"
"""نعمة المضادات الحيوية . تضع قط…",0,"""نعمة المضادات الحيوية تضع قطرة…"
"""الدودو جايه تكمل علي 💔""",0,"""الدودو جايه تكمل علي"""


In [65]:
X = df["cleaned_tweet"].to_numpy()
y = df["label"].to_numpy()

vectorizer = CountVectorizer()
X_vectorized = vectorizer.fit_transform(X)

In [70]:
vectorizer.vocabulary_

{'اعترف': 8175,
 'ان': 19376,
 'بتس': 22579,
 'كانو': 48435,
 'شوي': 40869,
 'يجيبو': 75821,
 'راسي': 36737,
 'لكن': 51599,
 'اليوم': 19081,
 'بالزايد': 21497,
 'توقعت': 31538,
 'اذا': 6779,
 'جات': 31899,
 'داريا': 35496,
 'بشوفهم': 24249,
 'كاملين': 48418,
 'بس': 23942,
 'لي': 53089,
 'للحين': 51969,
 'احس': 6217,
 'فيه': 47135,
 'احد': 6159,
 'ناقصهم': 59895,
 'avlu': 308,
 'الاهلي_الهلال': 10382,
 'اكتب': 8816,
 'توقعك': 31543,
 'لنتيجة': 52825,
 'لقاء': 51470,
 'الهلال': 18672,
 'والاهلي': 65069,
 'تحت': 27793,
 'التاق': 10980,
 'تحدي_اسرع_روقان': 27849,
 'وادخل': 64271,
 'في': 46978,
 'سحب': 38730,
 'قيمة': 48175,
 'ايفون': 20341,
 'على': 44107,
 'نعمة': 61030,
 'المضادات': 17444,
 'الحيوية': 12370,
 'تضع': 29523,
 'قطرةمضاد': 47798,
 'بنسلين': 25690,
 'بكتيريا': 25027,
 'فتنفجر': 45892,
 'تموت': 30938,
 'الأخيرة': 9123,
 'يبدو': 75169,
 'انها': 19885,
 'مقاومة': 58179,
 'فأخذ': 45176,
 'الدودو': 12813,
 'جايه': 32045,
 'تكمل': 30625,
 'علي': 44112,
 'أحببته': 2071,
 'حتى': 33315

In [79]:
tfidf = TfidfTransformer(use_idf=True, norm="l2", smooth_idf=True)
X_tfidf = tfidf.fit_transform(vectorizer.fit_transform(X)).toarray()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)